# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [1]:
import pandas as pd
import numpy as np
import math
import json
% matplotlib inline

# read in the json files
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Step 1: Load, Explore and Visualize Data

### Visualize DataFrames

In [2]:
portfolio

,channels,difficulty,duration,id,offer_type,reward
0,"[email, mobile, social]",10,7,ae264e3637204a6fb9bb56bc8210ddfd,bogo,10
1,"[web, email, mobile, social]",10,5,4d5c57ea9a6940dd891ad53e9dbe8da0,bogo,10
2,"[web, email, mobile]",0,4,3f207df678b143eea3cee63160fa8bed,informational,0
3,"[web, email, mobile]",5,7,9b98b8c7a33c4b65b9aebfe6a799e6d9,bogo,5
4,"[web, email]",20,10,0b1e1539f2cc45b7b9fa7c272da2e1d7,discount,5
5,"[web, email, mobile, social]",7,7,2298d6c36e964ae4a3e7e9706d1fb8c2,discount,3
6,"[web, email, mobile, social]",10,10,fafdcd668e3743c1bb461111dcafc2a4,discount,2
7,"[email, mobile, social]",0,3,5a8bc65990b245e5a138643cd4eb9837,informational,0
8,"[web, email, mobile, social]",5,5,f19421c1d4aa40978ebb69ca19b0e20d,bogo,5
9,"[web, email, mobile]",10,7,2906b810c7d4411798c6938adc9daaa5,discount,2


In [3]:
profile.head()

,age,became_member_on,gender,id,income
0,118,20170212,None,68be06ca386d4c31939f3a4f0e3dd783,NaN
1,55,20170715,F,0610b486422d4921ae7d2bf64640c50b,112000.0
2,118,20180712,None,38fe809add3b4fcf9315a9694bb96ff5,NaN
3,75,20170509,F,78afa995795e4d85b5d9ceeca43f5fef,100000.0
4,118,20170804,None,a03223e636434f42ac4c3df47e8bac43,NaN


In [4]:
transcript.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'}
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'}
2,offer received,e2127556f4f64592b11af22de27a7932,0,{'offer id': '2906b810c7d4411798c6938adc9daaa5'}
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'}
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'}


In [5]:
# Extracting offer id from the value column's dictionaries 
transcript_value = transcript[transcript['event'] != 'transaction']['value'].apply(pd.Series)
transcript_value = transcript_value.fillna('').drop('reward', axis=1)
transcript_value['offer_id'] = transcript_value['offer_id'] + transcript_value['offer id']
transcript_value = transcript_value.drop('offer id', axis=1)
transcript_value.head()

,offer_id
0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,2906b810c7d4411798c6938adc9daaa5
3,fafdcd668e3743c1bb461111dcafc2a4
4,4d5c57ea9a6940dd891ad53e9dbe8da0


In [6]:
# Assigning value column ONLY the offer id 
transcript['value'] = transcript_value
transcript_offers = transcript[transcript['event'] != 'transaction']
transcript_offers.head()

,event,person,time,value
0,offer received,78afa995795e4d85b5d9ceeca43f5fef,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,offer received,a03223e636434f42ac4c3df47e8bac43,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,offer received,e2127556f4f64592b11af22de27a7932,0,2906b810c7d4411798c6938adc9daaa5
3,offer received,8ec6ce2a7e7949b1bf142def7d0e0586,0,fafdcd668e3743c1bb461111dcafc2a4
4,offer received,68617ca6246f4fbc85e91a2a49552598,0,4d5c57ea9a6940dd891ad53e9dbe8da0


## Step 2: Clean and Pre-process Data

### Combine all 3 *.json* files into a single DataFrame and drop useless columns

In [40]:
df = transcript_offers.merge(profile, how='inner', left_on='person', right_on='id').drop(['person', 'id'], axis=1)
df = df.merge(portfolio, how='inner', left_on='value', right_on='id').drop(['value', 'id'], axis=1)
df = df.drop('became_member_on', axis=1)
df.head()

,event,time,age,gender,income,channels,difficulty,duration,offer_type,reward
0,offer received,0,75,F,100000.0,"[web, email, mobile]",5,7,bogo,5
1,offer viewed,6,75,F,100000.0,"[web, email, mobile]",5,7,bogo,5
2,offer completed,132,75,F,100000.0,"[web, email, mobile]",5,7,bogo,5
3,offer received,408,68,M,70000.0,"[web, email, mobile]",5,7,bogo,5
4,offer viewed,420,68,M,70000.0,"[web, email, mobile]",5,7,bogo,5


### Fill incomplete features  
Features: `gender`, `income`

In [41]:
# We can see that gender and income features both have nan values
df.isnull().sum()

event             0
time              0
age               0
gender        18776
income        18776
channels          0
difficulty        0
duration          0
offer_type        0
reward            0
dtype: int64

In [42]:
# Fill income nan values with mean of all incomes
df['income'].fillna(df['income'].mean(), inplace=True)

In [43]:
# Fill gender nan values with M or F 
fill_list = ['M', 'F']
random_series = pd.Series(np.random.choice(fill_list, size=len(df['gender'])))
df['gender'].fillna(random_series, inplace=True)

In [44]:
# Check that there are no more Nan values
df.isnull().sum()

event         0
time          0
age           0
gender        0
income        0
channels      0
difficulty    0
duration      0
offer_type    0
reward        0
dtype: int64

### Drop categorical features

In [45]:
df.drop(['gender', 'channels', 'offer_type'], axis=1, inplace=True)
df.head()

,event,time,age,income,difficulty,duration,reward
0,offer received,0,75,100000.0,5,7,5
1,offer viewed,6,75,100000.0,5,7,5
2,offer completed,132,75,100000.0,5,7,5
3,offer received,408,68,70000.0,5,7,5
4,offer viewed,420,68,70000.0,5,7,5


### Normalize numerical featuers
Features: `reward`, `difficulty`, `duation`, `age`, `income`, `time`

In [46]:
from sklearn.preprocessing import MinMaxScaler

numerical_features = ['time', 'age', 'income', 'difficulty', 'duration', 'reward']

scaler = MinMaxScaler((0, 1))
df_pca = pd.DataFrame(scaler.fit_transform(df[numerical_features].astype(float)))
df_pca.index = df.index
df_pca.columns = numerical_features
df_pca.head()

,time,age,income,difficulty,duration,reward
0,0.000000,0.57,0.777778,0.25,0.571429,0.5
1,0.008403,0.57,0.777778,0.25,0.571429,0.5
2,0.184874,0.57,0.777778,0.25,0.571429,0.5
3,0.571429,0.50,0.444444,0.25,0.571429,0.5
4,0.588235,0.50,0.444444,0.25,0.571429,0.5


In [47]:
# Ensure minimum value is 0 and maximum value is 1
df_pca.describe()

,time,age,income,difficulty,duration,reward
count,167581.000000,167581.000000,167581.000000,167581.000000,167581.000000,167581.000000
mean,0.495488,0.438626,0.404601,0.392520,0.515820,0.441991
std,0.277733,0.256932,0.225077,0.252447,0.305161,0.337336
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.235294,0.270000,0.244444,0.250000,0.285714,0.200000
50%,0.571429,0.400000,0.404601,0.500000,0.571429,0.500000
75%,0.714286,0.540000,0.533333,0.500000,0.571429,0.500000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [48]:
# Append scaled features to original DataFrame 
df[numerical_features] = df_pca
df.head()

,event,time,age,income,difficulty,duration,reward
0,offer received,0.000000,0.57,0.777778,0.25,0.571429,0.5
1,offer viewed,0.008403,0.57,0.777778,0.25,0.571429,0.5
2,offer completed,0.184874,0.57,0.777778,0.25,0.571429,0.5
3,offer received,0.571429,0.50,0.444444,0.25,0.571429,0.5
4,offer viewed,0.588235,0.50,0.444444,0.25,0.571429,0.5


### Map labels to 0 and 1
Label: `event`  
Map `offer received` and `offer viewed` to 0 and `offer completed` to 1

In [49]:
df.event.replace(['offer received', 'offer viewed', 'offer completed'], [0, 0, 1], inplace=True)
df.head()

,event,time,age,income,difficulty,duration,reward
0,0,0.000000,0.57,0.777778,0.25,0.571429,0.5
1,0,0.008403,0.57,0.777778,0.25,0.571429,0.5
2,1,0.184874,0.57,0.777778,0.25,0.571429,0.5
3,0,0.571429,0.50,0.444444,0.25,0.571429,0.5
4,0,0.588235,0.50,0.444444,0.25,0.571429,0.5


### Calculate percentage of *offers completed*:  
Formula: `# of offers completed / # of offers received`

In [50]:
num_offers_completed = df['event'].sum()
num_offers_received = sum(transcript['event'] == 'offer received')

print('Percentage of offers completed: {0:.2f}%'.format(num_offers_completed*100/num_offers_received))

Percentage of offers completed: 44.02%


### Calculate percentage of *transactions that involved offers*:  
Formula: `# of offers completed / (# of transactions - # of offers completed)`

In [51]:
num_transactions = sum(transcript['event'] == 'transaction')
print('Percentage of transactions that involved offers: {0:.2f}%'
      .format(num_offers_completed*100/(num_transactions - num_offers_completed)))

Percentage of transactions that involved offers: 31.87%


## Step 4: Split data into train/test datasets
Note: labels will be the `events` feature

In [52]:
# 70% of the data will be training, and 30% of the data will be testing 
train_fraction = 0.7

# Convert DataFrame to Numpy array
df_array = df.to_numpy()

# Set seed and shuffle data 
np.random.seed(1)
np.random.shuffle(df_array)

# Set training size 
train_size = int(df_array.shape[0] * train_fraction)

# Get training data 
train_x = df_array[:train_size, :-1]
train_y = df_array[:train_size, -1]

# Get testing data 
test_x = df_array[train_size:, :-1]
test_y = df_array[train_size:, -1]

print('Number of training points:', len(train_x))
print('Number of testing points:', len(test_x))

Number of training points: 117306
Number of testing points: 50275


## Step 5: Reduce dimensionality using PCA and find vectors of maximal variance

### Roles and Buckets

In [53]:
import sagemaker
from sagemaker import get_execution_role

session = sagemaker.Session()
role = get_execution_role()
bucket_name = session.default_bucket()

print("Role:", role)
print("Bucket Name:", bucket_name)

Role: arn:aws:iam::932663560021:role/service-role/AmazonSageMaker-ExecutionRole-20200401T133381
Bucket Name: sagemaker-ca-central-1-932663560021


### Define output location

In [54]:
# Location to store model artifacts
prefix = 'starbucks-capstone'

output_path = 's3://{}/{}/'.format(bucket_name, prefix)

print("Training artifacts will be uploaded to: {}".format(output_path))

Training artifacts will be uploaded to: s3://sagemaker-ca-central-1-932663560021/starbucks-capstone/


### Define a PCA Model

In [71]:
from sagemaker import PCA 

# Number of components will be 1 less than the total number of features 
num_components = len(df_pca.columns) - 1

pca_sagemaker = PCA(role=role,
                    train_instance_count=1,
                    train_instance_type='ml.c4.xlarge',
                    output_path=output_path,
                    num_components=num_components,
                    sagemaker_session=session)

### Convert data into a RecordSet format

In [72]:
# Convert DataFrame to Numpy array
train_data_np = df_pca.to_numpy().astype('float32')

# Convert to RecordSet format
train_data_formatted = pca_sagemaker.record_set(train_data_np)

### Train model

In [73]:
pca_sagemaker.fit(train_data_formatted)

2020-04-05 10:57:36 Starting - Starting the training job...
2020-04-05 10:57:37 Starting - Launching requested ML instances...
2020-04-05 10:58:33 Starting - Preparing the instances for training.........
2020-04-05 10:59:54 Downloading - Downloading input data
2020-04-05 10:59:54 Training - Downloading the training image..Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/05/2020 11:00:17 INFO 139945196414784] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-conf.json: {u'_num_gpus': u'auto', u'_log_level': u'info', u'subtract_mean': u'true', u'force_dense': u'true', u'epochs': 1, u'algorithm_mode': u'regular', u'extra_components': u'-1', u'_kvstore': u'dist_sync', u'_num_kv_servers': u'auto'}
[04/05/2020 11:00:17 INFO 139945196414784] Reading provided configuration from /opt/ml/input/config/hyperparameters.json: {u'feature_dim': u'6', u'mini_batch_size': u'500', u'num_components': 

### Unzip model details 

In [74]:
import boto3
import os

# Training job name
training_job_name = 'pca-2020-04-05-10-57-36-353'

# Location where model is saved 
model_key = os.path.join(prefix, training_job_name, 'output/model.tar.gz')
print(model_key)

# Download and unzip model
boto3.resource('s3').Bucket(bucket_name).download_file(model_key, 'model.tar.gz')

# Unzip model as model_algo-1
os.system('tar -zxvf model.tar.gz')
os.system('unzip model_algo-1')

starbucks-capstone/pca-2020-04-05-10-57-36-353/output/model.tar.gz


2304

### Access PCA model attributes by loading model artifacts using MXNet Array

In [75]:
import mxnet as mx

# Load unzipped model artifacts 
pca_model_params = mx.ndarray.load('model_algo-1')
print(pca_model_params)

{'s': 
[ 86.60691 109.72411 113.61226 131.10059 162.43192]
<NDArray 5 @cpu(0)>, 'v': 
[[-0.01073195  0.03829783  0.9978214  -0.04278264 -0.03065361]
 [-0.46515915  0.88418174 -0.03885758 -0.00940158  0.0157281 ]
 [ 0.88486266  0.46539485 -0.00894257 -0.00176188 -0.01714583]
 [-0.00832767  0.00500213 -0.02580628 -0.16115385 -0.5867276 ]
 [-0.01399932 -0.00633779 -0.04274958 -0.5889992  -0.5859688 ]
 [-0.01626555  0.00992087  0.01640357  0.790688   -0.55759335]]
<NDArray 6x5 @cpu(0)>, 'mean': 
[[0.49548814 0.43862584 0.4046013  0.39252004 0.5158196  0.44199103]]
<NDArray 1x6 @cpu(0)>}


### PCA model attributes

Three types of model attributes are contained within the PCA model:

- mean: The mean that was subtracted from a component in order to center it.
- v: The makeup of the principal components; (same as ‘components_’ in an sklearn PCA model).
- s: The singular values of the components for the PCA transformation. This does not exactly give the % variance from the original feature space, but can give the % variance from the projected feature space.

From s, we can get an approximation of the data variance that is covered in the first n principal components. The approximate explained variance is given by the formula: the sum of squared s values for all top n components over the sum over squared s values for all components:

$$\begin{equation*}
\frac{\sum_{n}^{ } s_n^2}{\sum s^2}
\end{equation*}$$

From v, we can learn more about the combinations of original features that make up each principal component.

In [76]:
# Get required parameters 
v = pd.DataFrame(pca_model_params['v'].asnumpy())
s = pd.DataFrame(pca_model_params['s'].asnumpy())

In [77]:
# Out of curiosity, check top 5 principal components
num_principal_components = 5

start_idx = num_components - num_principal_components
print(s.iloc[start_idx:, :])

            0
0   86.606911
1  109.724113
2  113.612259
3  131.100586
4  162.431915


### Calculate explained variance

In [54]:
# Keep top n principal components that account for at least 90% of the variance

### Examine component makeup

### Deploy PCA model

### Create a transformed DataFrame

### Define top *n* components to use in transformed data

### Deleting the Endopoint